# Create Strata for Activity Areas from LCC

In [36]:
# Standard Imports
import sys
import pandas as pd
import datetime

# Geospatial Imports
import geopandas as gpd
from shapely.geometry import Polygon
from geowrangler.validation import GeometryValidation

In [2]:
# Util imports
sys.path.append("../../../")  # include parent directory
from src.settings import GPKG_DATA_DIR, TMP_OUT_DIR

In [12]:
# Variables
VERSION = datetime.datetime.now().strftime("%Y%m%d")

# GCS Variables
LCC_GCS_DIR = "NULL"
STRATA_GCS_DIR = "gs://00_extract_vectors/"
SILUP_GCS_DIR = "gs://silup-gis/onebase/"

# Filepaths
LCC_FILEPATH = GPKG_DATA_DIR / "lcc_cadt.gpkg"
LCC_252_FILEPATH = TMP_OUT_DIR / "lcc_252.gpkg"
LCC_FINAL_FPATH = GPKG_DATA_DIR / "lcc_cadt_validated_20240812.gpkg"

# Load Data

Insert auto download of raster and converting raster to vector --  for now this loads a file processed using qgis

In [ ]:
lcc_all = gpd.read_file(LCC_FILEPATH)
lcc_252 = gpd.read_file(LCC_252_FILEPATH)

In [20]:
lcc = pd.concat([lcc_252, lcc_all])

## Validate geometries

In [ ]:
# check for invalid polygons
lcc[~lcc.geometry.is_valid].shape

(40584, 2)

In [22]:
# convert multipolygons to polygons
lcc = lcc.explode(index_parts=False)

In [24]:
# check and correct invalid polygons
lcc = GeometryValidation(lcc).validate_all()

In [25]:
lcc.geometry.type.unique()

array(['Polygon', 'MultiPolygon'], dtype=object)

In [26]:
# create separate gdf for multipolygons for 2nd level of explode and validation
lcc_poly = lcc[lcc["geometry"].type == "Polygon"].copy()
lcc_multi = lcc[lcc["geometry"].type == "MultiPolygon"].copy()

In [27]:
lcc_multi = lcc_multi.explode(index_parts=False)

In [28]:
lcc_multi.geometry.type.unique()

array(['Polygon'], dtype=object)

In [29]:
lcc_multi = GeometryValidation(lcc_multi).validate_all()

In [30]:
lcc_multi.geometry.type.unique()

array(['Polygon'], dtype=object)

In [32]:
# Combine the datasets again
lcc = pd.concat([lcc_poly, lcc_multi])

In [33]:
lcc[~lcc.geometry.is_valid].shape

(532, 7)

In [34]:
# buffer at 0m to remove invalid vertices
lcc["geometry"] = lcc["geometry"].buffer(0)

In [38]:
lcc[~lcc.geometry.is_valid].shape

(0, 7)

In [63]:
lcc.to_file(GPKG_DATA_DIR / "lcc_cadt_validated_20240812.gpkg")

# Generate Strata

In [ ]:
lcc = gpd.read_file(LCC_FINAL_FPATH)

In [46]:
from typing import Union
from shapely.geometry import MultiPolygon


def one_sided_poly_buffer(
    poly: Polygon, buffer_m: float
) -> Union[Polygon, MultiPolygon]:
    bound = poly.boundary
    # estimate how big the buffer zone will be
    poly_len = min(poly.length, buffer_m)
    buffer_area_estimate = buffer_m * poly_len
    # no need to buffer for small polygons
    if poly.area < buffer_area_estimate:
        buffered_bound = poly
    # buffer for sufficiently large polygons
    else:
        buffered_bound = bound.buffer(-buffer_m / 2, single_sided=True)
        # take the intersection to remove the outward spikes from a single sided buffer
        buffered_bound = buffered_bound.intersection(poly)
        # buffer it again to fill in inward spikes
        buffered_bound = buffered_bound.buffer(buffer_m / 2)
        # take intersection to remove outward spikes again
        buffered_bound = buffered_bound.intersection(poly)
    return buffered_bound

In [40]:
lcc_dipterocarp = lcc[lcc["DN"].isin([1, 4])]

In [42]:
lcc_dipterocarp.shape

(131809, 7)

In [43]:
test = lcc_dipterocarp[:100]

In [41]:
lc_swamp = lcc[lcc["DN"].isin([2, 5])]

In [52]:
# %%time
# buffered_diptercarp_1km = lcc_dipterocarp["geometry"].apply(
#     lambda x: one_sided_poly_buffer(x, -1_000)
# )

CPU times: user 1.23 s, sys: 5.89 ms, total: 1.24 s
Wall time: 1.24 s


In [64]:
lcc_dipterocarp["lines_geometry"] = lcc_dipterocarp["geometry"].boundary

In [61]:
lcc_dipterocarp

,DN,geometry,is_not_null,is_not_self_intersecting,is_oriented_properly,is_within_crs_bounds,area_is_not_zero
25,1,"POLYGON ((125.25111 8.98791, 125.25138 8.98791...",True,True,True,True,True
27,1,"POLYGON ((125.25273 8.98791, 125.25300 8.98791...",True,True,True,True,True
91,1,"POLYGON ((125.31157 8.98791, 125.31166 8.98791...",True,True,False,True,True
92,1,"POLYGON ((125.31220 8.98791, 125.31229 8.98791...",True,True,True,True,True
285,1,"POLYGON ((125.32397 8.98764, 125.32423 8.98764...",True,True,True,True,True
...,...,...,...,...,...,...,...
217668,1,"POLYGON ((126.21285 7.97372, 126.21285 7.97363...",True,True,True,True,True
217668,1,"POLYGON ((126.23567 7.95665, 126.23567 7.95674...",True,True,True,True,True
217668,1,"POLYGON ((126.22390 7.96554, 126.22444 7.96554...",True,True,True,True,True
217668,1,"POLYGON ((126.22174 7.96644, 126.22174 7.96653...",True,True,True,True,True
